In [1]:
using JuMP;
using CPLEX;

In [2]:
function extraire_array(str)
    matches = match(r"\[([\d,\s]+)\]", str)
# Если нашли совпадение, извлекаем числа и преобразуем их в массив
    if matches !== nothing
        p_values = parse.(Int64, split(matches.captures[1], ","))
        return p_values;
    end
end

extraire_array (generic function with 1 method)

In [3]:
function dataFiles(dirName::String) #ecrit tous les noms des fichoer qu ont .gr 
    names=[]
    for file in readdir(dirName)
        if occursin(".gr",file)
            push!(names,file)
        end
    end
    return names
end


dataFiles (generic function with 1 method)

In [4]:
function readfile(namefile::String)

    n=0;
    s=0;
    t=0;
    S=0;
    d1=0;
    d2=0;
    p=[];
    ph=[];
    flag = false;
    AdjMat=[];
    N=Dict{Int,Vector{Int}}()

    d=[];
    D=[];

    if isfile(namefile)
        file=open(namefile,"r")
        data=readlines(file);
        for line in data
            str=split(line," ")
            if (flag) ## on lit matrice d'adjeccance
                str[end]=chop(str[4])
                numbers = parse.(Int64,str[1:3]);
                d[numbers[1],numbers[2]]=numbers[3];
                AdjMat[numbers[1],numbers[2]]=1;
                push!(get!(N,numbers[1],[]),numbers[2])
                D[numbers[1],numbers[2]]=parse(Float64,str[4]);
            end
            if (str[1]=="n")
                n=parse(Int64,str[3]);
                d=zeros(Int64,n,n);
                D=zeros(Float64,n,n);
                AdjMat=zeros(Int64,n,n);
            end

            if (str[1]=="s")
                s=parse(Int64,str[3]);
            end
            
            if (str[1]=="t")
                t=parse(Int64,str[3]);
            end
            if (str[1]=="S")
                S=parse(Int64,str[3])
            end

            if (str[1]=="d1" )
                d1=parse(Int64,str[3]);
            end
            if (str[1]=="d2")
                d2=parse(Int64,str[3]);
            end
            
            if (str[1]=="p")
                p=extraire_array(join(str[3:end], " "))
            end

            if (str[1]=="ph")
                ph=extraire_array(join(str[3:end], " "))
            end

            if (str[1]=="Mat")
                flag= true;
            end
        end
    end
    return(n,s,t,S,d1,d2,p,ph,d,D,AdjMat)
end

readfile (generic function with 1 method)

In [ ]:
name = "20_USA-road-d.NY.gr"

In [ ]:
# parametres a definir n, s, t, S, d1,d2, p, ph
n,s,t,S,d1,d2,p,ph,d,D,AdjMat=readfile(name)

In [ ]:
# variable de base: x_ij, y_i 

@variable(m, x[1:n,1:n], Bin)
@variable(m, y[1:n],Bin)


# Contraint de base
@constraint(m,[i in 1:n, j in 1:n], x[i,j]<= y[j])
@constraint(m,[i in 1:n, j in 1:n], x[i,j]<= y[i])
@constraint(m,[i in 1:n, j in 1:n], x[i,j]<=AdjMat[i,j])
@constraint(m,sum(x[s,j] for j in 1:n)- sum(x[j,s] for j in 1:n) ==1)
@constraint(m,sum(x[t,j] for j in 1:n)- sum(x[j,t] for j in 1:n) == -1)

for i in 1:n
    if i != s && i != t
        @constraint(m, sum(x[i, j] for j in 1:n) - sum(x[j, i] for j in 1:n) == 0)
    end
end

## Variable par dualisation

@variable(m, a >=0)
@variable(m, b[1:n] >=0)
@variable(m, g>=0)
@variable(m, l[1:n,1:n] >=0)

#constraint dualisation

@constraint(m, [i in 1:n], a + b[i] >= y[i] * ph[i])
@constraint(m, [i in 1:n, j in 1:n], l[i,j]+g>= x[i,j] * d[i,j])
@constraint(m, sum(y[i]*p[i] + b[i]*2 for i in 1:n) + a*d2 <= S)

# obkectif function:
@objective(m, Min, sum(x[i,j]*d[i,j]+l[i,j]*D[i,j] for i in 1:n, j in 1:n) + g*d1 );

In [ ]:
# Résolution d’un modèle
optimize!(m)

# Récupération du status de la résolution
feasibleSolutionFound = primal_status(m) == MOI.FEASIBLE_POINT
isOptimal = termination_status(m) == MOI.OPTIMAL
if feasibleSolutionFound
    println("Probleme resolu:")
    #Valeur objectif
    println("Valeur de l'objectif : ",Jumodel_BC.objective_value(m))
    # Récupération des valeurs d’une variable
    vy = Jumodel_BC.value.(y)
    vg= Jumodel_BC.value(g)
    va = Jumodel_BC.value(a)
    vb=JuMP.value.(b)
    vX=JuMP.value.(x[:,:])
    vL=JuMP.value.(l[:,:])
   # print("y: ")
    #println(vy)
   # print("gamma: ")
    #println(vg)
    #print("alpha: ")
    #println(va)
end



In [24]:
## Resoulution par modele de prof:

function DualModel(n,s,t,S,d1,d2,p,ph,d,D,AdjMat)
    
    prof=Model(CPLEX.Optimizer)
    set_optimizer_attribute(prof, "CPX_PARAM_SCRIND", 0)
    set_optimizer_attribute(prof, "CPX_PARAM_TILIM", 300)
    # variable de base: w_ij, q_i 

    @variable(prof,x[1:n,1:n], Bin)
    @variable(prof, y[1:n],Bin)

    # Contraint de base
    @constraint(prof,[i in 1:n;i!=t], y[i]==sum(x[i,j] for j in 1:n if AdjMat[i,j]==1))
    @constraint(prof, y[t]==sum(x[i,t] for i in 1:n if AdjMat[i,t]==1))
    @constraint(prof,sum(x[s,j] for j in 1:n if AdjMat[s,j]==1)
        - sum(x[j,s] for j in 1:n if AdjMat[j,s]==1) ==1)
    @constraint(prof,sum(x[t,j] for j in 1:n if AdjMat[t,j]==1)
        - sum(x[j,t] for j in 1:n if AdjMat[j,t]==1) == -1)

    @constraint(prof, [i in 1:n;i!=s && i!=t], sum(x[i, j] for j in 1:n if AdjMat[i,j]==1) 
            - sum(x[j, i] for j in 1:n if AdjMat[j,i]==1) == 0)


    ## Variable par dualisation

    @variable(prof, a>=0)
    @variable(prof, b[1:n] >=0) # b
    @variable(prof, g>=0)
    @variable(prof, l[1:n,1:n] >=0) #l_ij

    #constraint dualisation

    @constraint(prof, [i in 1:n], a+ b[i] >= y[i]*ph[i])
    @constraint(prof, [i in 1:n, j in 1:n; AdjMat[i,j]==1], l[i,j]+g>= x[i,j] * d[i,j])
    @constraint(prof, sum(y[i]*p[i] + b[i]*2 for i in 1:n) + a*d2 <= S)

    # obkectif function:
    @objective(prof, Min, sum(x[i,j]*d[i,j]+l[i,j]*D[i,j] for i in 1:n, j in 1:n if AdjMat[i,j]==1) + g*d1 );

    optimize!(prof)

    # Récupération du status de la résolution
    feasibleSolutionFound = primal_status(prof) == MOI.FEASIBLE_POINT
    isOptimal = termination_status(prof) == MOI.OPTIMAL
    if feasibleSolutionFound
        println("Probleme resolu:")
        #Valeur objectif
        if (isOptimal)
            return JuMP.objective_value(prof),"opt";
        else
            return JuMP.objective_value(prof),"feas";
        end
    else
        return -1,"notfound"
    end

end


DualModel (generic function with 1 method)

In [25]:
file3="result_dual.csv"
files = [
    "1400_USA-road-d.COL.gr",
    "1500_USA-road-d.COL.gr",
    "1600_USA-road-d.COL.gr",
    "1700_USA-road-d.COL.gr",
    "1800_USA-road-d.COL.gr",
    "1900_USA-road-d.COL.gr",
    "2000_USA-road-d.COL.gr",
    "2100_USA-road-d.COL.gr",
    "2100_USA-road-d.NY.gr",
    "2200_USA-road-d.COL.gr",
    "2200_USA-road-d.NY.gr",
    "2300_USA-road-d.BAY.gr",
    "2300_USA-road-d.COL.gr",
    "2300_USA-road-d.NY.gr",
    "2400_USA-road-d.BAY.gr",
    "2400_USA-road-d.COL.gr",
    "2400_USA-road-d.NY.gr",
    "2500_USA-road-d.BAY.gr",
    "2500_USA-road-d.COL.gr",
    "2500_USA-road-d.NY.gr"
]

for name in files
    println(name)
    fout=open(file3,"a")
    # parametres a definir n, s, t, S, d1,d2, p, ph
    n,s,t,S,d1,d2,p,ph,d,D,AdjMat=readfile("Data_project\\data\\processed\\"*name)
    #println(n)
    start=time()
    v_dual,status=DualModel(n,s,t,S,d1,d2,p,ph,d,D,AdjMat)
    println(fout,name,",",v_dual,",",status,",",time()-start)
    close(fout)
end

1400_USA-road-d.COL.gr


Probleme resolu:
1500_USA-road-d.COL.gr


Probleme resolu:
1600_USA-road-d.COL.gr


Probleme resolu:
1700_USA-road-d.COL.gr


Probleme resolu:


...


2400_USA-road-d.COL.gr


2400_USA-road-d.NY.gr


2500_USA-road-d.BAY.gr
Probleme resolu:


2500_USA-road-d.COL.gr


2500_USA-road-d.NY.gr


### Plans coupant:
On resout Master Probleme, apres Sous-probleme1 , sous-probleme2, et on verifie la condition (9) et (10) (par rapport au rapport theorique). Si on trouve qu'au moins un des conditions ne sont pas verifier, on ajoute des coupes. Jusqu'a resodre tout le probleme.



In [6]:
#Master Probleme



function MP(n,s,t,S,d1,d2,p,ph,d,D,AdjMat)#return modele
    mp=Model(CPLEX.Optimizer)

    set_optimizer_attribute(mp, "CPX_PARAM_TILIM", 100)
    set_optimizer_attribute(mp, "CPX_PARAM_SCRIND", 0)

    @variable(mp,x[1:n ,1:n], Bin)
    @variable(mp,y[1:n], Bin)
    @variable(mp,z>=0)

    # Contraint de base
    @constraint(mp,[j in 1:n; j!=t], y[j]==sum( x[i,j] for i in 1:n if AdjMat[i,j]==1))
    @constraint(mp,y[t]==sum(x[j,t] for j in 1:n if AdjMat[j,t]==1))
    @constraint(mp,sum(x[s,j] for j in 1:n if AdjMat[s,j]==1) - sum(x[j,s] for j in 1:n if AdjMat[j,s]==1) ==1)
    @constraint(mp,sum(x[t,j] for j in 1:n if AdjMat[t,j]==1) - sum(x[j,t] for j in 1:n if AdjMat[j,t]==1) == -1)

    @constraint(mp, [i in 1:n;i != s && i != t], sum(x[i,j] for j in 1:n if AdjMat[i,j]== 1)
         - sum(x[j, i] for j in 1:n if AdjMat[j,i]==1) == 0)
      

    #Contrainte de Master Probleme
    @constraint(mp, sum(x[i,j]*d[i,j] for i in 1:n, j in 1:n if AdjMat[i,j]==1)<=z)
    @constraint(mp, sum(y[i]*p[i] for i in 1:n)<=S)

    # objectif function
    @objective(mp,Min,z)
    
    optimize!(mp)

    #println("Value:", JuMP.objective_value(mp))
    # Récupération du status de la résolution
    return mp;
    
end




MP (generic function with 1 method)

In [7]:
#return (obj_value, delta1, flag) - flag=1 si on a trouver la solution ot, 0 sinon
function SP1(x, n,s,t,S,d1,d2,p,ph,d,D,AdjMat)

    m1=Model(CPLEX.Optimizer)
    set_optimizer_attribute(m1, "CPX_PARAM_TILIM", 30)
    set_optimizer_attribute(m1, "CPX_PARAM_SCRIND", 0)
    #variable delta_ij
    @variable(m1,delta1[i in 1:n, j in 1:n;AdjMat[i,j]==1]>=0)

    #Contraintes
    @constraint(m1, [i in 1:n, j in 1:n; AdjMat[i,j]==1], delta1[i,j]<=D[i,j] )
    @constraint(m1,sum(delta1[i,j] for i in 1:n, j in 1:n if AdjMat[i,j]==1)<=d1)

    #Objectif
    @objective(m1,Max,sum(x[i,j]*d[i,j]*(1+delta1[i,j]) for i in 1:n, j in 1:n if AdjMat[i,j]==1))

    optimize!(m1)

    # Récupération du status de la résolution
    feasibleSolutionFound = primal_status(m1) == MOI.FEASIBLE_POINT
    isOptimal = termination_status(m1) == MOI.OPTIMAL
    if feasibleSolutionFound 
        #println("Sous-Probleme1 resolu:")
        #Valeur objectif
        #println("Valeur de l'objectif : ",JuMP.objective_value(m1))
        return (JuMP.objective_value(m1),JuMP.value.(delta1[:,:]),1)
    else
        return (-1,-1,0)
    end

end

SP1 (generic function with 1 method)

In [8]:
function SP2(y, n,s,t,S,d1,d2,p,ph,d,D,AdjMat)# return (objective_value, delta2, flag)

    m2=Model(CPLEX.Optimizer)
    set_optimizer_attribute(m2, "CPX_PARAM_TILIM", 10)
    set_optimizer_attribute(m2, "CPX_PARAM_SCRIND", 0)
    #variable delta_ij
    @variable(m2,delta2[i in 1:n]>=0)

    #Contraintes
    @constraint(m2, [i in 1:n], delta2[i]<=2);
    @constraint(m2,sum(delta2[i] for i in 1:n)<=d2)

    #Objectif
    @objective(m2,Max,sum(y[i]*(p[i]+delta2[i]*ph[i]) for i in 1:n));

    optimize!(m2)

    # Récupération du status de la résolution
    feasibleSolutionFound = primal_status(m2) == MOI.FEASIBLE_POINT
    isOptimal = termination_status(m2) == MOI.OPTIMAL
    if feasibleSolutionFound
        #println("Sous-Probleme1 resolu:")
        #Valeur objectif
        #println("Valeur de l'objectif : ",JuMP.objective_value(m2))
        return (JuMP.objective_value(m2),JuMP.value.(delta2),1)
    else
        return (-1,[],0)
    end
    
end

SP2 (generic function with 1 method)

In [9]:
function add_plan_coupant1!(model,_dlt1,n,s,t,S,d1,d2,p,ph,d,D,AdjMat)
   @constraint(model,model[:z] >= sum(d[i,j]*(_dlt1[i,j]+1)*(model[:x])[i,j] for i in 1:n, j in 1:n if AdjMat[i,j]==1))
    
end

function add_plan_coupant2!(model,_dlt2,n,s,t,S,d1,d2,p,ph,d,D,AdjMat)
    @constraint(model, S>= sum((model[:y])[i]*(_dlt2[i]*ph[i]+p[i]) for i in 1:n ))
    
end

add_plan_coupant2! (generic function with 1 method)

In [10]:
# While not find solution - ajoute plans coupants
filesNames=dataFiles("Data_project\\data\\processed\\")

# filesNames=[
#     "1100_USA-road-d.BAY.gr",
#     "1200_USA-road-d.BAY.gr",
#     "1300_USA-road-d.BAY.gr",
#     "1300_USA-road-d.COL.gr",
#     "1500_USA-road-d.BAY.gr",
#     "1600_USA-road-d.BAY.gr",
#     "1700_USA-road-d.BAY.gr",
#     "1800_USA-road-d.BAY.gr",
#     "1800_USA-road-d.COL.gr",
#     "1900_USA-road-d.BAY.gr",
#     "1900_USA-road-d.COL.gr",
#     "2000_USA-road-d.BAY.gr",
#     "2000_USA-road-d.NY.gr",
#     "2100_USA-road-d.BAY.gr",
#     "2100_USA-road-d.COL.gr",
#     "2100_USA-road-d.NY.gr",
#     "2200_USA-road-d.BAY.gr",
#     "2200_USA-road-d.NY.gr",
#     "2300_USA-road-d.BAY.gr",
#     "2300_USA-road-d.NY.gr",
#     "2400_USA-road-d.BAY.gr",
#     "2400_USA-road-d.COL.gr",
#     "2400_USA-road-d.NY.gr",
#     "2500_USA-road-d.BAY.gr",
#     "2500_USA-road-d.NY.gr"
# ]
resoudre=0
println("100 sec")
for name in filesNames
    println(name)
    #counter 
    k=0;
    # parametres a definir n, s, t, S, d1,d2, p, ph
    n,s,t,S,d1,d2,p,ph,d,D,AdjMat=readfile("Data_project\\data\\processed\\"*name)
    #println(n)
    mp=MP(n,s,t,S,d1,d2,p,ph,d,D,AdjMat)
    start=time()
    value_mp=0

    while true
        
        #Solve Master problem
        #println(k)
        optimize!(mp)
        
        feasibleSolutionFound = primal_status(mp) == MOI.FEASIBLE_POINT
        isOptimal = termination_status(mp) == MOI.OPTIMAL

        if time()-start >=100 || k>300
            println("Time expired")
            if feasibleSolutionFound
                if isOptimal
                    resoudre=resoudre+1
                    println(name*",",value_mp,",",k,",",time()-start,"opt")
                else
                    println(name*",",value_mp,",",k,",",time()-start,"feas")
                end
            else
                println(name*",",-1,",",k,",",time()-start,"nor found")
            end

            break
        end

        if feasibleSolutionFound
            #println("Probleme resolu:")
            #Valeur objectif
            #println("Valeur de l'objectif : ",JuMP.objective_value(mp))
            value_mp=JuMP.objective_value(mp);
            z_,x_,y_=value_mp, JuMP.value.(mp[:x][:,:]),JuMP.value.(mp[:y]);
        

        
            # Solve Sous-probleme 1
            z_1,delta1,flag1=SP1(x_, n,s,t,S,d1,d2,p,ph,d,D,AdjMat);

            z_2,delta2,flag2=SP2(y_, n,s,t,S,d1,d2,p,ph,d,D,AdjMat);

            if flag1!=1 || flag2!=1
                println("Don't know what to do")
            end

            # on ajoute epsilon 1e-5
            if (abs(z_1-z_)<=1e-5) && (z_2-1e-5<= S) 
                println("Solution optimal trouver: nb iterations ", k)
                println("Valeur opt: " ,z_);
                break; 
            else
                k+=1;
                if z_1!=z_ 
                    #println("Add plan coupans de type 1")
                    add_plan_coupant1!(mp,delta1,n,s,t,S,d1,d2,p,ph,d,D,AdjMat)
                end
                if (z_2> S)
                    #println("Add plan coupans de type 2")
                    add_plan_coupant2!(mp,delta2,n,s,t,S,d1,d2,p,ph,d,D,AdjMat)
                end
            end
        end

        
    end


end
 

100 sec


1000_USA-road-d.BAY.gr


## Callback; Branch and Cut 

isIntegerPoint (generic function with 1 method)

In [8]:
# On précise que le modèle doit utiliser notre fonction de callback

files = [
    "1100_USA-road-d.BAY.gr",
    "1200_USA-road-d.BAY.gr",
    "1300_USA-road-d.BAY.gr",
    "1300_USA-road-d.COL.gr",
    "1500_USA-road-d.BAY.gr",
    "1600_USA-road-d.BAY.gr",
    "1700_USA-road-d.BAY.gr",
    "1800_USA-road-d.BAY.gr",
    "1800_USA-road-d.COL.gr",
    "1900_USA-road-d.BAY.gr",
    "1900_USA-road-d.COL.gr",
    "2000_USA-road-d.BAY.gr",
    "2000_USA-road-d.NY.gr",
    "2100_USA-road-d.BAY.gr",
    "2100_USA-road-d.COL.gr",
    "2100_USA-road-d.NY.gr",
    "2200_USA-road-d.BAY.gr",
    "2200_USA-road-d.NY.gr",
    "2300_USA-road-d.BAY.gr",
    "2300_USA-road-d.NY.gr",
    "2400_USA-road-d.BAY.gr",
    "2400_USA-road-d.COL.gr",
    "2400_USA-road-d.NY.gr",
    "2500_USA-road-d.BAY.gr",
    "2500_USA-road-d.NY.gr"
]

for name in files
    foutBC=open("result_BC_2.csv","a")
    println(name)
    start=time()
    n,s,t,S,d1,d2,p,ph,d,D,AdjMat=readfile("Data_project\\data\\processed\\"*name)

    model_BC=Model(CPLEX.Optimizer)
    MOI.set(model_BC, MOI.NumberOfThreads(), 1)
    set_optimizer_attribute(model_BC, "CPX_PARAM_TILIM", 500)
    set_optimizer_attribute(model_BC, "CPX_PARAM_SCRIND", 0)


    function mon_callback(cb_data::CPLEX.CallbackContext, context_id::Clong)
         # On teste d’abord si c’est l’obtention d’une solution entière
         # qui a entraîné l’appel du callback
         # (cette fonction isIntegerPoint est définie ci-dessous mais son
         # contenu n’est pas très important)
     
         if isIntegerPoint(cb_data, context_id)
             # Cette ligne doit être appelée avant de pouvoir récupérer la
             # solution entière ayant entraîné l’appel du callback
             CPLEX.load_callback_variable_primal(cb_data, context_id)
             # On récupère la valeur de x
             x_val = callback_value.(cb_data, model_BC[:x][:,:])
             z_val = callback_value(cb_data, z)
             y_val = callback_value.(cb_data, model_BC[:y])
     
             z_1,delta1,flag1=SP1(x_val, n,s,t,S,d1,d2,p,ph,d,D,AdjMat);
     
             z_2,delta2,flag2=SP2(y_val, n,s,t,S,d1,d2,p,ph,d,D,AdjMat);
     
             if flag1!=1 || flag2!=1
                 println("Don't know what to do")
             end
     
             # on ajoute epsilon 1e-5
                 if abs(z_1-z_val)> 1e-5
                     cstr = @build_constraint(model_BC[:z] >= sum(d[i,j]*(delta1[i,j]+1)*(model_BC[:x])[i,j] for i in 1:n, j in 1:n if AdjMat[i,j]==1))
                     MOI.submit(model_BC, MOI.LazyConstraint(cb_data), cstr)
         
                     #println("Add constraint type 1")
                 end
                 if (z_2> S+1e-5)
                     cstr=@build_constraint(S>= sum((model_BC[:y])[i]*(delta2[i]*ph[i]+p[i]) for i in 1:n))
                     MOI.submit(model_BC,MOI.LazyConstraint(cb_data),cstr)
     
                    # println("Add constraint type 2")
                 end
             
         end
     end
     
     # Fonction permettant de déterminer si c’est l’obtention d’une
     # solution entière qui a entraîné l’appel d’un callback
     # (il n’est pas nécessaire d’en comprendre le fonctionnement)1
         function isIntegerPoint(cb_data::CPLEX.CallbackContext, context_id::Clong)
         # context_id == CPX_CALLBACKCONTEXT_CANDIDATE si le callback est
         # appelé dans un des deux cas suivants :
         # cas 1 - une solution entière a été obtenue; ou
         # cas 2 - une relaxation non bornée a été obtenue
         if context_id != CPX_CALLBACKCONTEXT_CANDIDATE
             return false
         end
         # Pour déterminer si on est dans le cas 1 ou 2, on essaie de récupérer la
         # solution entière courante
         ispoint_p = Ref{Cint}()
         ret = CPXcallbackcandidateispoint(cb_data, ispoint_p)
         # S’il n’y a pas de solution entière
         if ret != 0 || ispoint_p[] == 0
             return false
         else
             return true
         end
     end
             

    @variable(model_BC,x[1:n ,1:n], Bin)
    @variable(model_BC,y[1:n], Bin)
    @variable(model_BC,z>=0)

    # Contraint de base
    @constraint(model_BC,[j in 1:n; j!=t], y[j]==sum( x[i,j] for i in 1:n if AdjMat[i,j]==1))
    @constraint(model_BC,y[t]==sum(x[j,t] for j in 1:n if AdjMat[j,t]==1))
    @constraint(model_BC,sum(x[s,j] for j in 1:n if AdjMat[s,j]==1) - sum(x[j,s] for j in 1:n if AdjMat[j,s]==1) ==1)
    @constraint(model_BC,sum(x[t,j] for j in 1:n if AdjMat[t,j]==1) - sum(x[j,t] for j in 1:n if AdjMat[j,t]==1) == -1)

    @constraint(model_BC, [i in 1:n;i != s && i != t],sum(x[i,j] for j in 1:n if AdjMat[i,j]== 1)
         - sum(x[j, i] for j in 1:n if AdjMat[j,i]==1) == 0)
    

    #Contrainte de Master Probleme
    @constraint(model_BC, sum(x[i,j]*d[i,j] for i in 1:n, j in 1:n if AdjMat[i,j]==1)<=z)
    @constraint(model_BC, sum(y[i]*p[i] for i in 1:n)<=S)

    # objectif function
    @objective(model_BC,Min,z)

    MOI.set(model_BC, CPLEX.CallbackFunction(), mon_callback)
    optimize!(model_BC)

    feasibleSolutionFound = primal_status(model_BC) == MOI.FEASIBLE_POINT
    isOptimal = termination_status(model_BC) == MOI.OPTIMAL
    if feasibleSolutionFound
         print(foutBC,name,",")
         if isOptimal
              print(foutBC,JuMP.objective_value(model_BC),",optimal,")
         #Valeur objectif
         else
              print(foutBC,JuMP.objective_value(model_BC),",feasible,")
         end
   else
       print(foutBC,name,",-1,not find,")
    end
    println(foutBC,time()-start)
    close(foutBC)
end

1100_USA-road-d.BAY.gr


1200_USA-road-d.BAY.gr


1300_USA-road-d.BAY.gr


1300_USA-road-d.COL.gr


1500_USA-road-d.BAY.gr


1600_USA-road-d.BAY.gr


1700_USA-road-d.BAY.gr

1800_USA-road-d.BAY.gr
1800_USA-road-d.COL.gr


## Heuristique

On va procéder de la façon suivante:
on commence en s, apres on regarde tout les voisin possible , au debut on choisi tels i dans le N(s), qui satisfait: y_i*p_i<=S; comme ca on forme un ensemble C(s); pour chaque v in C(s) on calcule solution de la relaxation en imposent s-i comme le premier arete. Ainsi de suite, jusqu’a arriver en t

In [ ]:
function heuristique()

    C_i=[]# voisin de i qui satisfait y_i*p_i<=S
    Eval_candidat=Dict{Int, Float64}()
    sommets_vu=[]

    # Remplir C_i
    for v in N(s)
        if p[v]<=S
            push!(C_i,v)
            # on resou la relax ou on impose v==1
            setvalue.(model[:x][s, v], 1)
            setvalue.(model[:y][v],1)

            optimize!(model)

            if termination_status(model)==MOI.OPTIMAL
                Eval_candidat[v]=objective_value(model)
            end
        end
    end

    sort!(collect(Eval_candidat),by= x->x[2])

    push!(sommets_vu,keys(first(Eval_candidat)))
    empty!(Eval_candidat)

    while()



end

In [13]:
name="20_USA-road-d.BAY.gr"
n,s,t,S,d1,d2,p,ph,d,D,AdjMat=readfile("Data_project\\data\\processed\\"*name)
    #println(n)

model=Model(CPLEX.Optimizer)
set_optimizer_attribute(model, "CPX_PARAM_SCRIND", 0)    
#Variables
@variable(model,x[i in 1:n,j in 1:n;AdjMat[i,j]==1],Bin)
@variable(model,y[1:n],Bin)

#Constraintes
@constraint(model,sum(p[i]*y[i] for i in 1:n)<=S)
@constraint(model,[i in 1:n;i!=t], y[i]==sum(x[i,j] for j in 1:n if AdjMat[i,j]==1))
@constraint(model,y[t]==sum(x[i,t] for i in 1:n if AdjMat[i,t]==1))
@constraint(model,[i in 1:n;i!=s&&i!=t], 
        sum(x[j,i] for j in 1:n if AdjMat[j,i]==1) - sum(x[i,j] for j in 1:n if AdjMat[i,j]==1)==0)
@constraint(model,sum(x[s,j] for j in 1:n if AdjMat[s,j]==1) - sum(x[j,s] for j in 1:n if AdjMat[j,s]==1)==1)
@constraint(model,sum(x[t,j] for j in 1:n if AdjMat[t,j]==1) - sum(x[j,t] for j in 1:n if AdjMat[j,t]==1)==-1)

#objectif
@objective(model,Min,sum(d[i,j]*x[i,j] for i in 1:n,j in 1:n if AdjMat[i,j]==1))

optimize!(model)

# Récupération du status de la résolution
feasibleSolutionFound = primal_status(model) == MOI.FEASIBLE_POINT
    isOptimal = termination_status(model) == MOI.OPTIMAL
    if feasibleSolutionFound
        println("Probleme resolu:")
        #Valeur objectif
        if isOptimal 
            println("Valeur de l'objectif : ",JuMP.objective_value(model))
            stat_solution_y=JuMP.value.(y)
            stat_solution_x=JuMP.value.(x[:,:])
        end
    end


best_find=JuMP.objective_value(model)
vu_somets=[]
uses_sommet=findall(stat_solution_y .==1)
non_vu_somets=uses_sommet
println(non_vu_somets)

println(stat_solution_y)
sol_opt_x=stat_solution_x
sol_opt_y=stat_solution_y;

start=time()

while true

    if time()-start >500
        break
    end

    newSommet=false
    v=rand(non_vu_somets)
    pres_v = findall(stat_solution_x[:,v] .==1)
    
    suc_v=  findall(stat_solution_x[v,:] .==1)
    deleteat!(non_vu_somets,v)
    sommet=-1;
    
    for u in N[pres_v] 
        if suc_v in N[u]
            # esseyon resoudre pour ce chemin avec u:

            sol_new_y=sol_opt_y
            sol_new_y[v]=0;
            sol_new_y[u]=1;
            sol_new_x=sol_opt_x;
            sol_new_x[pres_v,v]=0
            sol_new_x[v,suc_v]=0
            sol_new_x[u,suc_v]=1
            sol_new_x[pres_v,u]=1
            mh=Model()
            set_optimizer_attribute(mh, "CPX_PARAM_TILIM", 200)
            set_optimizer_attribute(mh, "CPX_PARAM_SCRIND", 0)
            
            # variable de base: w_ij, q_i 

            @variable(mh,x[1:n,1:n],Bin)
            @variable(mh, y[1:n],Bin)



            # Contraint de base
            @constraint(mh,[j in 1:n;j!=t], y[j]==sum(x[j,i] for i in 1:n if AdjMat[j,i]==1))
            @constraint(mh, y[t]==sum(w[i,t] for i in 1:n if AdjMat[i,t]==1))
            @constraint(mh,sum(x[s,j] for j in 1:n if AdjMat[s,j]==1)- sum(x[j,s] for j in 1:n if AdjMat[j,s]==1) ==1)
            @constraint(mh,sum(x[t,j] for j in 1:n if AdjMat[t,j]==1)- sum(x[j,t] for j in 1:n if AdjMat[j,t]==1) == -1)

            @constraint(mh,[i in 1:n;i != s && i != t], sum(x[i, j] for j in 1:n if AdjMat[i,j]==1)
                    - sum(x[j, i] for j in 1:n if AdjMat[j,i]==1) == 0)
            

            ## Variable par dualisation

            @variable(mh, t2>=0)
            @variable(mh, b[1:n] >=0) # z'_v
            @variable(mh, t1>=0)
            @variable(mh, z[1:n,1:n] >=0) #z_ij

            #constraint dualisation

            @constraint(mh, [i in 1:n], t2 + b[i] >= y[i] * ph[i])
            @constraint(mh, [i in 1:n, j in 1:n; AdjMat[i,j]==1], z[i,j]+t1>= x[i,j] * d[i,j])
            @constraint(mh, sum(y[i]*p[i] + b[i]*2 for i in 1:n) + t2*d2 <= S)

            # obkectif function:
            @objective(mh, Min, sum(x[i,j]*d[i,j]+z[i,j]*D[i,j] for i in 1:n, j in 1:n if AdjMat[i,j]==1) + t1*d1 );

            JuMP.fix(x,sol_new_x)
            JuMP.fix(y,sol_new_y)

            optimize!(mh)

            feasibleSolutionFound = primal_status(mh) == MOI.FEASIBLE_POINT
            isOptimal = termination_status(mh) == MOI.OPTIMAL

            if feasibleSolutionFound
                if isOptimal && JuMP.objective_value(mh)<best_find
                    best_find=val(sol_new)
                    sol_opt=sol_new
                    newSommet=true
                    sommet=u;
                    break
                else JuMP.objective_value(mh)<best_find
                        best_find=val(sol_new)
                        sol_opt=sol_new
                        newSommet=true
                        sommet=u;
                        break
                end
            end
        end

    end

    if newSommet==true 
        push!(non_vu_somets,sommet)
    end

end





Probleme resolu:


Valeur de l'objectif : 9365.0


[12, 13, 15, 17, 19]


[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0]


ErrorException: `Base.size` is not implemented for `SparseAxisArray` because although it is a subtype of `AbstractArray`, it is conceptually closer to a dictionary with `N`-dimensional keys. If you encounter this error and you didn't call `size` explicitly, it is because you called a method that is unsupported for `SparseAxisArray`s. Consult the JuMP documentation for a list of supported operations.

In [7]:
A=[0 1 0; 0 0 1; 0 0 0]

A[findall(A[:,2] .==1),:]


1×3 Matrix{Int64}:
 0  1  0